# Generating 3D mass using GAN
This notebook is used to generate voxel 3d models trained on tower dataset. These voxel models are post processed with blender for further enhancement of the final shape mass.

##Importing usefull libraries

In [ ]:
!pip install voxelfuse
!pip install tensorflow-gan

import os
import scipy
import scipy.io
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, Conv3D, Conv3DTranspose, Activation, BatchNormalization, LeakyReLU, Flatten, Reshape, Dense, Dropout, UpSampling3D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import backend 
from tensorflow.keras.constraints import Constraint
import tensorflow_gan as tfgan
from google.colab import output
from voxelfuse.voxel_model import VoxelModel
from voxelfuse.mesh import Mesh
from voxelfuse.primitives import generateMaterials
from scipy.ndimage import zoom

#Implementation based on original paper https://arxiv.org/abs/1707.09557

output.clear()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Building Generator


In [ ]:
def make_Generator(kernel_size=5, strides=2, latent_dim=32):
     model = Sequential()
     model.add(Input(shape=(latent_dim,)))

     model.add(Dense(units=2048, input_shape=(latent_dim,), use_bias=False))
     model.add(BatchNormalization(momentum=0.9))
     model.add(Reshape((2, 2, 2, 256)))
     model.add(Activation('relu'))
     #model.add(Reshape((2, 2, 2, 256)))
     model.add(Dropout(0.4))

     model.add(UpSampling3D())
     model.add(Conv3DTranspose(filters=512, kernel_size=kernel_size, padding='same', use_bias=False))
     model.add(BatchNormalization(momentum=0.9))
     model.add(Activation('relu'))

     model.add(UpSampling3D())
     model.add(Conv3DTranspose(filters=256, kernel_size=kernel_size, padding='same', use_bias=False))
     model.add(BatchNormalization(momentum=0.9))
     model.add(Activation('relu'))

     model.add(UpSampling3D())
     model.add(Conv3DTranspose(filters=128, kernel_size=kernel_size, padding='same', use_bias=False))
     model.add(BatchNormalization(momentum=0.9))
     model.add(Activation('relu'))

     model.add(UpSampling3D())
     model.add(Conv3DTranspose(filters=64, kernel_size=kernel_size, padding='same', use_bias=False))
     model.add(BatchNormalization(momentum=0.9))
     model.add(Activation('relu'))

     model.add(UpSampling3D())
     model.add(Conv3DTranspose(filters=1, kernel_size=kernel_size, padding='same', use_bias=False))
     model.add(Activation('sigmoid'))

     return model

## Building Discriminator

In [ ]:
def make_Discriminator(kernel_size=5, strides=2, im_dim=64):
    #const = ClipConstraint(0.01)

    model = Sequential()
    model.add(Input(shape=(im_dim, im_dim, im_dim, 1)))
    
    model.add(Conv3D(filters=64, kernel_size=kernel_size, strides=strides, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.35))

    model.add(Conv3D(filters=128,kernel_size=kernel_size, strides=strides, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.35))

    model.add(Conv3D(filters=256,kernel_size=kernel_size, strides=strides, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.35))

    model.add(Conv3D(filters=512,kernel_size=kernel_size, strides=strides, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.35))

    model.add(Flatten())
    model.add(Dense(1))
    #model.add(Activation('sigmoid'))

    return model

## Preparing data for training

In [ ]:
training_data = []

for i, filename in enumerate(os.listdir('/content/gdrive/MyDrive/AI_Artathon/tower_dataset_npz')):
  print(i, filename)
  voxel_model = np.load(f'/content/gdrive/MyDrive/AI_Artathon/tower_dataset_npz/{filename}')['arr_0']
  new_array = zoom(voxel_model, (0.5, 0.5, 0.5)) # change size of dataset to fit the model
  training_data.append(new_array)

training_data = np.array(training_data).astype(np.float32)

0 166bd86229867093031b562a9def0c1_surface.npz
1 1683c5bc839969f438cd8eb23853c009_surface.npz
2 15cc3d9020384e8d6e09a8e31c7575c5_solid.npz
3 166bd86229867093031b562a9def0c1_solid.npz
4 10312d9d07db5d9a4159aeb47682f2cb_surface.npz
5 1683c5bc839969f438cd8eb23853c009_solid.npz
6 10be0ff0e5eb12584edd82d34d189d0d_surface.npz
7 10be0ff0e5eb12584edd82d34d189d0d_solid.npz
8 10312d9d07db5d9a4159aeb47682f2cb_solid.npz
9 149f13db5c358ecfde1c31612629d3dc_solid.npz
10 149f13db5c358ecfde1c31612629d3dc_surface.npz
11 15cc3d9020384e8d6e09a8e31c7575c5_surface.npz
12 224934a28705403238cd8eb23853c009_surface.npz
13 1b781ad3315b36510f233dcbf8432d5_surface.npz
14 17f5482dd8b9dbddc850800ac472a578_solid.npz
15 19e66890128da2d629647c2672d3167a_surface.npz
16 17e229dbb42786e6b164f257e05e5435_solid.npz
17 1fc436fd53db26d667ff3be08608de4d_solid.npz
18 283c8fd32d9b9a95c2c6f10e262c6c44_solid.npz
19 1ebb17d0790e384844e2f9503fe415b5_surface.npz
20 1ec4566d10cc58d7a2f3410d5eb66b3d_surface.npz
21 26e099d5d7040b03b96aa5

In [ ]:
training_data[0].shape

(64, 64, 64)

In [ ]:
file_list = os.listdir('/content/gdrive/MyDrive/AI_Artathon/tower_dataset_npz')
len(file_list)

266

In [ ]:
print(file_list[0])

853.npz


##Defining usefull functions

In [ ]:
def save_voxel(voxel_model, path):
    model = VoxelModel(voxel_model)  
    mesh = Mesh.fromVoxelModel(model)
    mesh.export(path)

In [ ]:
def gradient_penalty(real, fake, epsilon): 
    global discriminator
    #mixed_images = real * epsilon + fake * (1 - epsilon)
    mixed_images = fake + epsilon * (real - fake)
    with tf.GradientTape() as tape:
        tape.watch(mixed_images) 
        mixed_scores = discriminator(mixed_images)
        

    gradient = tape.gradient(mixed_scores, mixed_images)[0]
    
    gradient_norm = tf.norm(gradient)
    penalty = tf.math.reduce_mean((gradient_norm - 1)**2)
    return penalty

def generator_loss(fake_output):
    gen_loss = -1. * tf.math.reduce_mean(fake_output)
    return gen_loss

def discriminator_loss(real_output, fake_output, gradient_penalty):
    c_lambda = 10
    loss = tf.math.reduce_mean(fake_output) - tf.math.reduce_mean(real_output) + c_lambda * gradient_penalty
    return loss

##Main training function

In [ ]:
discriminator = make_Discriminator()
generator = make_Generator()

def train(sample_path, checkpoints_path, num_epochs=1000, batch_size=8, latent_dim=32, restore_D=None, restore_G=None, restore_epoch=0):
    global discriminator
    global generator

    sample_epoch = 1000
    save_epoch = 1000

    dis_optim = RMSprop(lr=0.0002, decay=6e-8)
    gen_optim = RMSprop(lr=0.0001, decay=3e-8)

    #discriminator = make_Discriminator()

    if restore_D!=None:
      discriminator=tf.keras.models.load_model(restore_D)

    #generator = make_Generator()

    if restore_G!=None:
      generator=tf.keras.models.load_model(restore_G)

    print('Generator')
    generator.summary()
    generator.compile(optimizer=gen_optim)
    #z = Input(shape=(latent_dim,))
    #img = generator(z)

    #discriminator.trainable = False
    #validity = discriminator(img)

    #combined = Model(inputs=z, outputs=validity)
    #combined.compile(loss='binary_crossentropy', optimizer=gen_optim, metrics=['accuracy'])
    #print('Combined...')
    #combined.summary()

    #discriminator.trainable = True
    discriminator.compile(optimizer=dis_optim)
    print('Discriminator...')
    discriminator.summary()

    # load data
    # data_loader = DataLoader(args)
    # X_train = np.array(data_loader.load_data()).astype(np.float32)
    dl, gl = [],[]
    for epoch in range(restore_epoch, num_epochs):
        #sample a random batch
        idx = np.random.randint(len(training_data), size=batch_size)
        # print('Sampling indices...' + str(idx))
        real = training_data[idx]
        real = real.reshape(real.shape+(1,))

        noise = tf.random.normal([batch_size, latent_dim])

        #training discriminator 3 times for each batch
        for i in range(3):

          with tf.GradientTape() as disc_tape:
        
            generated_images = generator(noise, training=True)

            real_output = discriminator(real, training=True)
            fake_output = discriminator(generated_images, training=True)
        
            epsilon = tf.random.normal([batch_size, 1, 1, 1, 1], 0.0, 1.0)
        
            gp = gradient_penalty(real, generated_images, epsilon)
        
            disc_loss = discriminator_loss(real_output, fake_output, gp)

    
          gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
          dis_optim.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        with tf.GradientTape() as gen_tape:
          generated_images = generator(noise, training=True)
          fake_output = discriminator(generated_images, training=True)
          gen_loss = generator_loss(fake_output)
        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gen_optim.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

        print('Training epoch {}/{}, d_loss: {},  g_loss: {}'.format(epoch+1, num_epochs, disc_loss, gen_loss))

        # sampling
        if epoch % sample_epoch == 0:
            if not os.path.exists(sample_path):
                os.makedirs(sample_path)
            print('Sampling...')
            sample_noise = np.random.uniform(-1.0, 1.0, size=[1, latent_dim]).astype(np.float64)
            voxel_model = generator.predict(sample_noise, verbose=1)
            voxel_model = voxel_model.reshape(voxel_model[0].shape[:-1])
            voxel_model = np.rint(voxel_model)
            try:
              save_voxel(voxel_model, sample_path + f'/epoch_{epoch+1}.obj')
            except:
              print('Could not create voxel model... Continuing training')

        # save weights
        if epoch % save_epoch == 0:
            if not os.path.exists(checkpoints_path):
                os.makedirs(checkpoints_path)
            generator.save(checkpoints_path + '/generator_epoch_' + str(epoch+1))
            discriminator.save(checkpoints_path + '/discriminator_epoch_' + str(epoch+1))

In [ ]:
sample_path = '/content/gdrive/MyDrive/AI_Artathon/voxel_model/samplings'
checkpoints_path = '/content/gdrive/MyDrive/AI_Artathon/voxel_model/model_checkpoints'

train(sample_path, checkpoints_path, num_epochs=1000000)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Generator
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 2048)              65536     
                                                                 
 batch_normalization_15 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                                 
 reshape_3 (Reshape)         (None, 2, 2, 2, 256)      0         
                                                                 
 activation_18 (Activation)  (None, 2, 2, 2, 256)      0         
                                                                 
 dropout_19 (Dropout)        (None, 2, 2, 2, 256)      0         
                                                                 
 up_sampling3d_15 (UpSamplin  (None, 4, 4, 4, 256)     0         
 g3D)                                       

Meshing: 100%|██████████| 1082/1082 [00:00<00:00, 5941.44it/s]


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/AI_Artathon/voxel_model/model_checkpoints/generator_epoch_1/assets
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/AI_Artathon/voxel_model/model_checkpoints/discriminator_epoch_1/assets
Training epoch 2/1000000, d_loss: -276.4544372558594,  g_loss: -268.59686279296875
Training epoch 3/1000000, d_loss: -48.90522384643555,  g_loss: -5.487723350524902
Training epoch 4/1000000, d_loss: -66.7907485961914,  g_loss: 86.01617431640625
Training epoch 5/1000000, d_loss: -215.08181762695312,  g_loss: -113.71449279785156
Training epoch 6/1000000, d_loss: -235.7108154296875,  g_loss: -181.21682739257812
Training epoch 7/1000000, d_loss: -102.75450134277344,  g_loss: 53.26789855957031
Training epoch 8/1000000, d_loss: -125.82190704345703,  g_loss: 47.15963363647461
Training epoch 9/1000000, d_loss: -45.089271545410156,  g_loss: 79.95674133300781
Training epoch 10/1000000, d_loss: -43.33488464355469,  g_loss: 98.82160949707031
Trai

Finding exterior voxels: 100%|██████████| 64/64 [00:00<00:00, 139.50it/s]
Meshing: 0it [00:00, ?it/s]


...

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/AI_Artathon/voxel_model/model_checkpoints/generator_epoch_15001/assets
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/AI_Artathon/voxel_model/model_checkpoints/discriminator_epoch_15001/assets
Training epoch 15002/1000000, d_loss: -2023.4454345703125,  g_loss: -644.3487548828125
Training epoch 15003/1000000, d_loss: 84.99811553955078,  g_loss: -38.72496795654297
Training epoch 15004/1000000, d_loss: 24.000028610229492,  g_loss: -174.74356079101562
Training epoch 15005/1000000, d_loss: 243.61044311523438,  g_loss: -178.62391662597656
Training epoch 15006/1000000, d_loss: -164.54443359375,  g_loss: -29.436840057373047
Training epoch 15007/1000000, d_loss: -473.4492492675781,  g_loss: -63.45062255859375
Training epoch 15008/1000000, d_loss: 282.6832275390625,  g_loss: -209.2794189453125
Training epoch 15009/1000000, d_loss: 191.1538543701172,  g_loss: -349.5321044921875
Training epoch 15010/1000000, d_loss: 45.53947448

Meshing: 100%|██████████| 15887/15887 [00:04<00:00, 3462.68it/s]


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/AI_Artathon/voxel_model/model_checkpoints/generator_epoch_16001/assets
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/AI_Artathon/voxel_model/model_checkpoints/discriminator_epoch_16001/assets
Training epoch 16002/1000000, d_loss: -41.772552490234375,  g_loss: 1.987135887145996
Training epoch 16003/1000000, d_loss: -85.11634826660156,  g_loss: 32.976627349853516
Training epoch 16004/1000000, d_loss: -1201.23095703125,  g_loss: -79.84915161132812
Training epoch 16005/1000000, d_loss: -59.34614562988281,  g_loss: -53.68952941894531
Training epoch 16006/1000000, d_loss: -396.5933532714844,  g_loss: -70.68522644042969
Training epoch 16007/1000000, d_loss: -74.70321655273438,  g_loss: 0.5682721138000488
Training epoch 16008/1000000, d_loss: -39.82909393310547,  g_loss: -5.4286789894104
Training epoch 16009/1000000, d_loss: -363.9013671875,  g_loss: -297.58282470703125
Training epoch 16010/1000000, d_loss: -3.781394958496

Meshing: 100%|██████████| 697/697 [00:00<00:00, 5086.59it/s]


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/AI_Artathon/voxel_model/model_checkpoints/generator_epoch_17001/assets
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/AI_Artathon/voxel_model/model_checkpoints/discriminator_epoch_17001/assets
Training epoch 17002/1000000, d_loss: -113.71065521240234,  g_loss: -4.394912242889404
Training epoch 17003/1000000, d_loss: -119.52833557128906,  g_loss: 4.6538004875183105
Training epoch 17004/1000000, d_loss: -29.8170166015625,  g_loss: 36.74287796020508
Training epoch 17005/1000000, d_loss: -115.60608673095703,  g_loss: 236.50270080566406
Training epoch 17006/1000000, d_loss: -204.8673858642578,  g_loss: 16.32577896118164
Training epoch 17007/1000000, d_loss: -111.50860595703125,  g_loss: -46.21577453613281
Training epoch 17008/1000000, d_loss: -95.96099853515625,  g_loss: -87.69141387939453
Training epoch 17009/1000000, d_loss: -89.70298767089844,  g_loss: -21.78197479248047
Training epoch 17010/1000000, d_loss: -69.54600

##Generating random samples


In [ ]:
def generate_samples(G_path, out_path, num=100, latent_dim=32):
  if not os.path.exists(out_path):
        os.makedirs(out_path)
  G = tf.keras.models.load_model(G_path)
  z = np.random.normal(0.0, 1.0, size=[num, latent_dim])
  voxel_models = G.predict(z, verbose=1)
  voxel_models = voxel_models.reshape(voxel_models.shape[:-1])
  voxel_models = np.rint(voxel_models)
  for i, model in enumerate(voxel_models):
    if(model.sum()<2500): continue
    save_voxel(model, f'{out_path}/sample_{i}.obj')

In [ ]:
generate_samples('/content/gdrive/MyDrive/AI_Artathon/voxel_model/model_checkpoints/generator_epoch_17001', '/content/gdrive/MyDrive/AI_Artathon/voxel_model/out/17002', num=50)

2/2 [==============================] - 9s 3s/step


Meshing: 100%|██████████| 2974/2974 [00:00<00:00, 6655.70it/s]
